In [146]:
from surianalytics.connectors import ESQueryBuilder, escape as es_escape
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import plotly.express as px
import mercury as mr
import ipywidgets as widgets
import json
from IPython.display import display
from django.utils import timezone
from datetime import timedelta, timezone as dt_tz
from django.conf import settings

try:
    # allow to use timezone
    settings.configure()
    settings.USE_TZ = True
except:
    pass

# disable insecure warning
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# Be careful with max_rows, too much rows leads to freeze browser/os
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)


# Set time range for the data
global_from_date='2021-06-16T16:33:54Z'
global_to_date='2021-06-16T19:29:09Z'

def flatten_aggregation(content, keys):
    def recurse_go(val, res, keys, i, row_res):
        for item in val.get('buckets', []):
            if i == 0:
                row_res = {}
            row_res[i] = item['key']
            if str(i + 2) in item:
                recurse_go(item[str(i + 2)], res, keys, i + 1, row_res)
            else:
                for j in range(0, i + 1):
                    res[keys[j]].append(row_res[j])
                res[keys[i + 1]].append(item['doc_count'])

    res = {}
    for key in keys:
        res[key] = []
    i = 0
    for key, val in content.get('aggregations', {}).items():
        recurse_go(val, res, keys, i, {})
    pd_struct = {}
    for key in keys:
        pd_struct[key] = res[key]
    return pd.DataFrame(pd_struct)
        


In [147]:
# Get flow top talker
builder = ESQueryBuilder()
builder.set_index('logstash-flow-*')
builder.set_page_size(0)

qfilter = 'event_type: flow'
builder.set_qfilter(qfilter)

builder.add_aggs('src_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('dest_ip.keyword', order='_count', sort='desc', size=10)

builder.set_from_date(global_from_date)
builder.set_to_date(global_to_date)

r = builder.post()
content = r.json()

keys = ['Source', 'Destination', 'Count']
res = flatten_aggregation(content, keys)

df = res.sort_values('Count',ascending=False)
df

,Source,Destination,Count
0,10.6.15.119,65.60.35.141,28288
1,10.6.15.119,10.6.15.5,802
10,10.6.15.187,10.6.15.5,612
20,10.6.15.93,10.6.15.5,540
2,10.6.15.119,194.226.60.15,156
11,10.6.15.187,207.246.77.75,48
3,10.6.15.119,10.6.15.255,44
12,10.6.15.187,10.6.15.255,40
32,fe80:0000:0000:0000:1c01:b22b:e837:eee2,ff02:0000:0000:0000:0000:0000:0000:00fb,38
30,10.6.15.1,224.0.0.251,38


In [148]:
# Cipher suite count on degraded / insecure
builder = ESQueryBuilder()
builder.set_index('logstash-http-*')
builder.set_page_size(0)

qfilter = 'event_type: http'
builder.set_qfilter(qfilter)

builder.add_aggs('src_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('dest_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('http.hostname.keyword', order='_count', sort='desc', size=10)

builder.set_from_date(global_from_date)
builder.set_to_date(global_to_date)

r = builder.post()
content = r.json()

keys = ['Source', 'Destination', 'HTTP Host', 'Count']
res = flatten_aggregation(content, keys)

df = res.sort_values('Count',ascending=False)
df

,Source,Destination,HTTP Host,Count
0,10.6.15.119,65.60.35.141,65.60.35.141,14146
1,10.6.15.119,194.226.60.15,hadevatjulps.com,78
10,10.6.15.93,23.47.49.179,download.windowsupdate.com,30
2,10.6.15.119,23.47.49.143,download.windowsupdate.com,6
17,10.6.15.187,23.47.49.156,download.windowsupdate.com,6
20,10.6.15.187,72.21.81.240,ctldl.windowsupdate.com,3
19,10.6.15.187,23.47.48.70,ctldl.windowsupdate.com,3
18,10.6.15.187,13.107.4.50,download.windowsupdate.com,3
13,10.6.15.93,23.47.49.139,download.windowsupdate.com,3
4,10.6.15.119,8.209.119.208,srand04rf.ru,3


In [149]:
# Get flow top talker
builder = ESQueryBuilder()
builder.set_index('logstash-flow-*')
builder.set_page_size(0)
builder.set_from_date(global_from_date)
builder.set_to_date(global_to_date)

qfilter = 'event_type: flow AND dest_ip:"10.6.15.0/24"'
builder.set_qfilter(qfilter)

builder.add_aggs('dest_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('proto.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('dest_port', order='_count', sort='desc', size=10)
builder.add_aggs('app_proto.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('src_ip.keyword', order='_count', sort='desc', size=10)

r = builder.post()
content = r.json()

keys = ['Server', 'Proto', 'Port', 'App', 'Client', 'Count']
res = flatten_aggregation(content, keys)

df = res.groupby(['Server', 'Proto', 'Port', 'App']).agg({'Client' : ','.join, 'Count' : 'sum'})  
df

Client  \
Server      Proto Port  App                                                    
10.6.15.187 TCP   445   smb                                      10.6.15.119   
            UDP   137   failed                                     10.6.15.5   
10.6.15.255 UDP   137   failed  10.6.15.1,10.6.15.119,10.6.15.187,10.6.15.93   
                  138   failed  10.6.15.119,10.6.15.187,10.6.15.93,10.6.15.1   
10.6.15.5   TCP   88    krb5              10.6.15.119,10.6.15.187,10.6.15.93   
                  135   dcerpc            10.6.15.119,10.6.15.187,10.6.15.93   
                  389   failed            10.6.15.187,10.6.15.119,10.6.15.93   
                  445   smb               10.6.15.119,10.6.15.187,10.6.15.93   
                  49674 dcerpc            10.6.15.119,10.6.15.187,10.6.15.93   
            UDP   53    dns               10.6.15.119,10.6.15.187,10.6.15.93   
                  123   ntp               10.6.15.119,10.6.15.187,10.6.15.93   
                  138   failed                                   10.6.15.187   
                  389   failed            10.6.15.187,10.6.15.119,10.6.15.93   
10.6.15.93  TCP   445   smb                                      10.6.15.119   

                                Count  
Server      Proto Port  App            
10.6.15.187 TCP   445   smb         2  
            UDP   137   failed      2  
10.6.15.255 UDP   137   failed     28  
                  138   failed    102  
10.6.15.5   TCP   88    krb5      142  
                  135   dcerpc    158  
                  389   failed    170  
                  445   smb        94  
                  49674 dcerpc    158  
            UDP   53    dns      1010  
                  123   ntp        66  
                  138   failed      2  
                  389   failed    126  
10.6.15.93  TCP   445   smb         2

In [167]:
# Get flow top talker
builder = ESQueryBuilder()
builder.set_index('logstash-http-*')
builder.set_page_size(0)
builder.set_from_date(global_from_date)
builder.set_to_date(global_to_date)

qfilter = 'event_type: http AND src_ip:"10.6.15.0/24"'
builder.set_qfilter(qfilter)

builder.add_aggs('http.http_user_agent.keyword', order='_count', sort='desc', size=20)
builder.add_aggs('http.user_agent.name.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('http.user_agent.version.keyword', order='_count', sort='desc', size=10)

r = builder.post()
content = r.json()

keys = ['Agent', 'Browser', 'Version', 'Count']
res = flatten_aggregation(content, keys)

df = res.sort_values('Count',ascending=False)
display(df)

builder.nb_aggs = 0
builder.aggs = []

builder.add_aggs('http.http_user_agent.keyword', order='_count', sort='asc', size=20)
builder.add_aggs('http.user_agent.name.keyword', order='_count', sort='asc', size=10)
builder.add_aggs('http.user_agent.version.keyword', order='_count', sort='asc', size=10)

r = builder.post()
content = r.json()
res = flatten_aggregation(content, keys)

df = res.sort_values('Count',ascending=True)
display(df)

,Agent,Browser,Version,Count
0,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727),IE,8.0,13991
1,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30),IE,7.0,155
2,Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko,IE,11.0,82
3,Windows-Update-Agent/10.0.10011.16384 Client-Protocol/2.31,Windows-Update-Agent,10.0.10011.16384,48
4,Microsoft-CryptoAPI/10.0,Microsoft-CryptoAPI,10.0,14
5,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36 Edg/91.0.864.48",Edge,91.0.864.48,6
6,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E),IE,11.0,1
7,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; WOW64; Trident/5.0),IE,9.0,1
8,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; MATP; MATP),IE,9.0,1


,Agent,Browser,Version,Count
0,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E),IE,11.0,1
1,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; WOW64; Trident/5.0),IE,9.0,1
2,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; MATP; MATP),IE,9.0,1
3,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36 Edg/91.0.864.48",Edge,91.0.864.48,6
4,Microsoft-CryptoAPI/10.0,Microsoft-CryptoAPI,10.0,14
5,Windows-Update-Agent/10.0.10011.16384 Client-Protocol/2.31,Windows-Update-Agent,10.0.10011.16384,48
6,Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko,IE,11.0,82
7,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30),IE,7.0,155
8,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727),IE,8.0,13991
